<center>TP noté 1: 3M 

Pierrick Bournez
</center>

Helper : https://scikit-image.org/docs/stable/auto_examples/applications/plot_morphology.html#sphx-glr-auto-examples-applications-plot-morphology-py

## Preliminaire
On charge une image de test : 

In [ ]:
!pip install scikit-image

In [ ]:
import matplotlib.pyplot as plt
from skimage import data
from skimage.util import img_as_ubyte
import numpy as np

orig_phantom = img_as_ubyte(data.shepp_logan_phantom())
fig, ax = plt.subplots()
ax.imshow(orig_phantom, cmap=plt.cm.gray)
plt.title("Original image")

## Question 1 :
Implantez la dilatation/erosion par un ES horizontal ou vertical de longueur fixe.

2- Réalisez ces opérations en utilisant la bibliothèque Numpy

dans toute la suite on suppose que le point de calcul est centré sur l'élément structurant

On implémente la dilatation/érosion horizontal

In [ ]:
def operation_horizontal(input_img: np.array, ES: np.array, op="dilation"):
    n, m = input_img.shape
    center = int(ES.shape[1] / 2)  # Center of the SE element
    output = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            min_ind = max(0, j - center)
            max_ind = min(m, j + center + 1)
            if op == "dilation":
                value = -1
            if op == "erosion":
                value = np.inf
            for k in range(min_ind, max_ind):
                if op == "dilation":
                    if input_img[i, k] > value:
                        value = input_img[i, k]
                if op == "erosion":
                    if input_img[i, k] < value:
                        value = input_img[i, k]
            output[i, j] = value

    return output

In [ ]:
bright_square = np.array(
    [
        [0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0],
    ],
    dtype=np.uint8,
)
from skimage.morphology import footprint_rectangle

res = operation_horizontal(bright_square, footprint_rectangle((1, 3)), op="dilation")
plt.imshow(res, cmap=plt.cm.gray)
plt.title("Dilated Bright square with a 1,3 horizontal SE")

On calcule la dilation vertical de la même manière


In [ ]:
def operation_vertical(input_img: np.array, ES: np.array, op="dilation"):
    n, m = input_img.shape
    center = int(ES.shape[0] / 2)  # Center of the SE element
    output = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            min_ind = max(0, i - center)
            max_ind = min(n - 1, i + center + 1)
            if op == "dilation":
                value = -1
            if op == "erosion":
                value = np.inf
            for k in range(min_ind, max_ind):
                if op == "dilation":
                    if input_img[k, j] > value:
                        value = input_img[k, j]
                if op == "erosion":
                    if input_img[k, j] < value:
                        value = input_img[k, j]

            output[i, j] = value

    return output

In [ ]:
res = operation_vertical(bright_square, footprint_rectangle((3, 1)), op="dilation")
plt.imshow(res, cmap=plt.cm.gray)
plt.title("Dilated Bright square with a 1,3 vertical SE")

On peut maintenant coder la  Dilatation par un rectangle 

In [ ]:
def operation_naive(input_img: np.array, ES: np.array, op="dilation"):
    row = ES[0, :]
    # create a shape (,) of row
    row = row.reshape((1, row.shape[0]))
    column = ES[:, 0]
    result = operation_horizontal(input_img, row, op=op)
    result = operation_vertical(result, column, op=op)
    return result

In [ ]:
res = operation_naive(bright_square, footprint_rectangle((2, 2)), op="dilation")
plt.imshow(res, cmap=plt.cm.gray)
plt.title("Dilated Bright square with a 2,2 horizontal SE,colors are inverted ")

On vérifie que le code marche pour l'image d'origine

On vérifie que le code donne des résultats cohérents :


    - Pour les opérations "verticales"

In [ ]:
import matplotlib.pyplot as plt

# Assuming `operation_vertical` and `footprint_rectangle` are defined

plt.figure(figsize=(12, 6))

# First image (Vertical dilation)
plt.subplot(1, 2, 1)  # Create subplot (1 row, 2 columns, 1st subplot)
plt.imshow(operation_vertical(orig_phantom, footprint_rectangle((10, 1))), cmap="gray")
plt.title("Vertical Dilation for a 10 vertical SE")
plt.axis("off")  # Hide axes for better visualization

# Second image (Vertical erosion)
plt.subplot(1, 2, 2)  # Create subplot (1 row, 2 columns, 2nd subplot)
plt.imshow(
    operation_vertical(orig_phantom, footprint_rectangle((10, 1)), op="erosion"),
    cmap="gray",
)
plt.title("Vertical Erosion for a 10 vertical SE")
plt.axis("off")  # Hide axes for better visualization

# Display the images
plt.tight_layout()

plt.show()

    - Pour les opérations horizontales

In [ ]:
import matplotlib.pyplot as plt

# Assuming `operation_vertical` and `footprint_rectangle` are defined

plt.figure(figsize=(12, 6))

# First image (Vertical dilation)
plt.subplot(1, 2, 1)  # Create subplot (1 row, 2 columns, 1st subplot)
plt.imshow(
    operation_horizontal(orig_phantom, footprint_rectangle((1, 10))), cmap="gray"
)
plt.title("horizontal Dilation for a 10 horizontal bar")
plt.axis("off")  # Hide axes for better visualization

# Second image (horizontal erosion)
plt.subplot(1, 2, 2)  # Create subplot (1 row, 2 columns, 2nd subplot)
plt.imshow(
    operation_horizontal(orig_phantom, footprint_rectangle((1, 10)), op="erosion"),
    cmap="gray",
)
plt.title("horizontal Erosion for a 10 horizontal bar")
plt.axis("off")  # Hide axes for better visualization

# Display the images
plt.tight_layout()

plt.show()

    - Pour les rectangles

In [ ]:
import matplotlib.pyplot as plt

# Assuming `operation_vertical` and `footprint_rectangle` are defined

plt.figure(figsize=(12, 6))

# First image (Vertical dilation)
plt.subplot(1, 2, 1)  # Create subplot (1 row, 2 columns, 1st subplot)
plt.imshow(operation_naive(orig_phantom, footprint_rectangle((10, 10))), cmap="gray")
plt.title("horizontal Dilation with a 10,10 rectangle Structuring Element ")
plt.axis("off")  # Hide axes for better visualization

# Second image (horizontal erosion)
plt.subplot(1, 2, 2)  # Create subplot (1 row, 2 columns, 2nd subplot)
plt.imshow(
    operation_naive(orig_phantom, footprint_rectangle((10, 10)), op="erosion"),
    cmap="gray",
)
plt.title("horizontal Erosion for a 10,10 rectangle Structuring Element")
plt.axis("off")  # Hide axes for better visualization

# Display the images
plt.tight_layout()

plt.show()

# Question 2 
Implantez la dilatation et l'érosion efficaces par l'algorithme de van Herk (page 160 du poly de 2018)


( voir See polycopié and this one https://pismin.com/10.1016/0167-8655(92)90069-c)



On code Van Herk d'abord en 1D 

In [ ]:
def dilation_Vanherk_1D(input_img_1D: np.array, ES: np.array):
    k = ES.shape[0]
    n_ini = input_img_1D.shape[0]
    # if n%k !=0 pad it with +Inf
    if n_ini % k != 0:
        input_img_1D = np.concatenate(
            (input_img_1D, np.array([-np.inf] * (k - n_ini % k)))
        )
        n = input_img_1D.shape[0]
    else:
        n = n_ini

    g = np.zeros(n)
    h = np.zeros(n)
    for i in range(0, n, k):
        subarray = input_img_1D[i : i + k]
        int_result_g = np.zeros(k)
        int_result_h = np.zeros(k)

        int_result_g[0] = subarray[0]
        int_result_h[-1] = subarray[-1]
        for o in range(1, k):
            int_result_g[o] = max(int_result_g[o - 1], subarray[o])
        for o in range(k - 2, -1, -1):
            int_result_h[o] = max(int_result_h[o + 1], subarray[o])

        g[i : i + k] = int_result_g
        h[i : i + k] = int_result_h

    # compute the result

    result = np.zeros(n)
    for x in range(n):
        min_ind = int(max(0, x - (k - 1) / 2))
        max_ind = int(min(n - 1, x + (k - 1) / 2))
        result[x] = max(g[max_ind], h[min_ind])

    # remove the padded value if needed
    result = result[:n_ini]
    return result


bright_line = np.array([0, 1, 0, 0, 0, 0, 0, 1, 1])
ES = np.array([1, 1, 1])
dilation_Vanherk_1D(bright_line, ES)

We code now the Van Herk dilation for a rectangle

In [ ]:
def dilation_Vanherk(input_img: np.array, ES: np.array):
    n, m = input_img.shape
    sample_row = ES[0, :]
    sample_column = ES[:, 0]
    result = np.zeros((n, m))
    for i in range(n):
        result[i, :] = dilation_Vanherk_1D(input_img[i, :], sample_row)
    for j in range(m):
        result[:, j] = dilation_Vanherk_1D(result[:, j], sample_column)
    return result

In [ ]:
bright_square = np.array(
    [
        [0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0],
    ],
    dtype=np.uint8,
)
res = dilation_Vanherk(bright_square, footprint_rectangle((1, 3)))
plt.imshow(res, cmap=plt.cm.gray)
plt.title("Dilated Bright square with a 1,3 horizontal SE with Van Herk")

In [ ]:
def erosion_Vanherk_1D(input_img_1D: np.array, ES: np.array):
    k = ES.shape[0]
    n_ini = input_img_1D.shape[0]
    # if n%k !=0 pad it with +Inf
    if n_ini % k != 0:
        input_img_1D = np.concatenate(
            (input_img_1D, np.array([np.inf] * (k - n_ini % k)))
        )
        n = input_img_1D.shape[0]
    else:
        n = n_ini

    g = np.zeros(n)
    h = np.zeros(n)
    for i in range(0, n, k):
        subarray = input_img_1D[i : i + k]
        int_result_g = np.zeros(k)
        int_result_h = np.zeros(k)

        int_result_g[0] = subarray[0]
        int_result_h[-1] = subarray[-1]
        for o in range(1, k):
            int_result_g[o] = min(int_result_g[o - 1], subarray[o])

        for o in range(k - 2, -1, -1):
            int_result_h[o] = min(int_result_h[o + 1], subarray[o])

        g[i : i + k] = int_result_g
        h[i : i + k] = int_result_h

    result = np.zeros(n)
    for x in range(n):
        max_ind = int(min(n - 1, x + (k - 1) / 2))
        min_ind = int(max(0, x - (k - 1) / 2))

        value_g = g[max_ind]
        value_h = h[min_ind]
        if x - (k - 1) / 2 < 0:
            value_h = np.inf
        if x + (k - 1) / 2 >= n:
            value_g = np.inf

        result[x] = min(value_g, value_h)

    # remove the padded value if needed
    result = result[:n_ini]
    return result


# Example usage
bright_line = np.array([1, 1, 0, 1, 1, 1, 1])
ES = np.array([1, 1, 1])
result = erosion_Vanherk_1D(bright_line, ES)
print("Erosion result:", result)

In [ ]:
import numpy as np


def erosion_Vanherk(input_img: np.array, ES: np.array):
    n, m = input_img.shape
    sample_row = ES[0, :]
    sample_column = ES[:, 0]
    result = np.zeros((n, m))
    for i in range(n):
        result[i, :] = erosion_Vanherk_1D(input_img[i, :], sample_row)
    for j in range(m):
        result[:, j] = erosion_Vanherk_1D(result[:, j], sample_column)
    return result

It matches our initial Implementation !

On vérifie que le résultat est cohérent avec notre première implémentation

In [ ]:
import matplotlib.pyplot as plt

# Assuming `operation_vertical` and `footprint_rectangle` are defined

plt.figure(figsize=(12, 6))

# First image (Vertical dilation)
plt.subplot(1, 2, 1)  # Create subplot (1 row, 2 columns, 1st subplot)
plt.imshow(
    dilation_Vanherk(input_img=orig_phantom, ES=footprint_rectangle((10, 10))),
    cmap="gray",
)
plt.title("Dilatation avec Van Herk pour un rectangle 10,10")
plt.axis("off")  # Hide axes for better visualization
plt.show()
plt.imshow(operation_naive(orig_phantom, footprint_rectangle((10, 10))), cmap="gray")
plt.title(" Dilatation naive pour un rectangle 10,10")


plt.show()


# Assuming `operation_vertical` and `footprint_rectangle` are defined

plt.figure(figsize=(12, 6))

# First image (Vertical dilation)
plt.subplot(1, 2, 1)  # Create subplot (1 row, 2 columns, 1st subplot)
plt.imshow(
    erosion_Vanherk(input_img=orig_phantom, ES=footprint_rectangle((10, 10))),
    cmap="gray",
)
plt.title("erosion avec Van Herk pour un rectangle 10,10")
plt.axis("off")  # Hide axes for better visualization
plt.show()
plt.imshow(
    operation_naive(orig_phantom, footprint_rectangle((10, 10)), op="erosion"),
    cmap="gray",
)
plt.title(" erosion naive pour un rectangle 10,10")

from skimage.morphology import erosion

plt.figure()
plt.imshow(erosion(orig_phantom, footprint_rectangle((10, 10))), cmap="gray")
plt.title("erosion avec skimage pour un rectangle 10,10")

# Question 3 
 Comparez les temps de calcul par comparaison avec l'implantation classique, par décomposition et par van Herk. Comparez à Scikit-image.

In [ ]:
from skimage.morphology import dilation as dilation_skimage

In [ ]:
import time


def benchmark_op(operation, input_img, ES):
    start_time = time.time()
    result = operation(input_img, ES)
    end_time = time.time()
    # print("Time taken for the operation",operation," is ",end_time-start_time)
    return result, end_time - start_time

We benchmark our bright square and the orig_phantom

In [ ]:
result = {}
footprint_to_assess = {
    " 1,3": footprint_rectangle((1, 3)),
    "5,5": footprint_rectangle((5, 5)),
    "20,20": footprint_rectangle((20, 20)),
    "50,50": footprint_rectangle((50, 50)),
}
for op in [dilation_skimage, dilation_Vanherk, operation_naive]:
    result[op.__name__] = {}
    for key in footprint_to_assess.keys():
        print("key", key)
        print(op)
        result[op.__name__][key] = []
        for k in range(10):
            result[op.__name__][key].append(
                benchmark_op(op, orig_phantom, footprint_to_assess[key])[1]
            )

In [ ]:
# rename the dilation key by dilation_skimage
result["dilation_skimage"] = result.pop("dilation")

In [ ]:
# Plotting the results
fig, ax = plt.subplots(figsize=(10, 6))
footprint_sizes = ["1,3", "5,5", "20,20", "50,50"]
colors = ["blue", "green", "red"]
operations = list(result.keys())

for idx, op in enumerate(operations):
    means = [np.mean(result[op][key]) for key in result[op].keys()]
    stds = [np.std(result[op][key]) for key in result[op].keys()]
    # Line plot
    ax.plot(footprint_sizes, means, label=op, linestyle="-", marker="o", alpha=0.8)

    # Shaded area for standard deviation
    ax.fill_between(
        footprint_sizes,
        [m - s for m, s in zip(means, stds)],
        [m + s for m, s in zip(means, stds)],
        alpha=0.2,
    )

ax.set_title("Benchmark Results For Dilation with different ES  different Rectangle")
ax.set_xlabel("Footprint Size")
ax.set_ylabel("Execution Time (s)")
ax.legend(title="Operations")

plt.tight_layout()
plt.show()

Quelle que soit le rectangle considéré, l'algorithme de skimage est meilleur que l'algorithme de Van Herk que l'algorithme naif.

La différence entre Van Herk et l'algorithme naif est "logique" en raison de la meilleur compléxité.

je  suspecte que l'algorithme de Van Herk est moins bien que l'algorithme de skimage car le code de skimage est codé en C++ au contraire de notre algorithme.

# 5- Implantez les opérateurs efficaces avec des ES diagonaux.
En faite , cela revient à appliquer le Van Herk 1D sur chaque diagonale de la matrice d'entrée !

In [ ]:
def Van_herk_diag(input_img, k):
    # k is tb he size of the SE element
    #! TODO
    n, m = input_img.shape
    result = np.zeros((n, m))
    for offset in range(-n + 1, n):  # Covers all diagonals
        diagonal = np.diagonal(input_img, offset=offset)
        result_diag = dilation_Vanherk_1D(diagonal, np.ones(k))
        # Write the correct diagonal placement, didn't find a easy way with diagonal.
        if offset >= 0:
            for i in range(len(result_diag)):
                result[i + offset, i] = result_diag[i]
        elif offset < 0:
            for i in range(len(result_diag)):
                result[i, i - offset] = result_diag[i]
    return result

# 6- Implantez les opérateurs efficaces avec des octogones.



D'abord on définit un exemple

In [ ]:
# create a black image with one white square on the middle

n = 20
m = 20
img = np.zeros((n, m))
img[n // 2, m // 2] = 1
plt.imshow(img, cmap="gray")
plt.title("Input of the Van Herk Octogone")

In [ ]:
def dilate_by_octagon(img, length):
    """
    Perform dilation using an octagonal structuring element by combining
    horizontal, vertical, and diagonal dilations.

    Parameters:
    - img: Input image.
    - length: Length of the structuring element.

    Returns:
    - dilated: The image dilated by the octagonal structuring element.
    """
    # Horizontal dilation
    footprint = footprint_rectangle((1, length))
    dilated = dilation_Vanherk(img, footprint)

    # Vertical dilation (transpose for reuse)
    img_transposed = dilated.T
    footprint = footprint_rectangle((1, length))

    dilated_ver = dilation_Vanherk(img_transposed, footprint)
    dilated = dilated_ver.T

    # Diagonal dilations
    dilated = Van_herk_diag(dilated, length)
    dilated = Van_herk_diag(np.fliplr(dilated), length)  # Diagonal (/)
    dilated = np.fliplr(dilated)

    return dilated


result = dilate_by_octagon(img, 5)

In [ ]:
plt.imshow(img, cmap="gray")
plt.title("Input of the Van Herk Octogone")
plt.show()

plt.imshow(result, cmap="gray")
plt.title("Output of the Van Herk Octogone")